# Mid-Term Project

In [4]:
## Load packages
%pip install ucimlrepo
from ucimlrepo import fetch_ucirepo 


Note: you may need to restart the kernel to use updated packages.


In [2]:
# fetch dataset 
breast_cancer = fetch_ucirepo(id=14) 
  
# data (as pandas dataframes) 
X = breast_cancer.data.features 
y = breast_cancer.data.targets 
  
# metadata 
print(breast_cancer.metadata) 
  
# variable information 
print(breast_cancer.variables) 

TypeError: 'NoneType' object is not iterable